In [3]:
import numpy as np
import h5py
import os

In [17]:
log_base = '/om/user/dzd123/logs/498/'
files = ['enc_test_save',
    'enc_obj3_save',
    'enc_obj9_save'
        ]
encs_and_ys = []
for file in files:
    f = np.load(log_base + file + '.npz')
    encs_and_ys += [(f['enc'], f['y'])]

In [22]:
encs_and_ys[0]

(array([[[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [ -1.86190009e-03,  -2.25162506e-03,   9.35071707e-03, ...,
            1.49554014e-03,   4.89182770e-04,   1.21057034e-03],
         [  7.76872039e-05,  -7.82907009e-04,   6.15826994e-03, ...,
            1.66033208e-03,   1.23801827e-03,   5.52594662e-04],
         [ -1.28377974e-03,  -1.96942687e-03,   6.94327056e-03, ...,
            1.42711401e-03,   5.10446727e-04,   6.65724277e-04],
         [  2.27459520e-03,   6.46403432e-03,  -1.12410560e-02, ...,
           -3.00586224e-03,  -6.97068870e-04,   7.60436058e-04],
         [  1.46660209e-03,   4.98083234e-03,  -8.59658420e-03, ...,
           -2.75948644e-03,  -8.33019614e-04,   7.06374645e-04]],
 
        [[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  9.73574817e-04,   3.91274691e-03,  -7.76994228e

In [18]:
num_points = 
f = h5py.File('/om/user/dzd123/physics_b.h5')

In [21]:
list(f.keys())

['col',
 'col_long',
 'col_mass32',
 'col_obj3',
 'col_obj9',
 'obs_x',
 'obs_x_long',
 'obs_x_mass32',
 'obs_x_obj3',
 'obs_x_obj9',
 'ro_x',
 'ro_x_long',
 'ro_x_mass32',
 'ro_x_obj3',
 'ro_x_obj9',
 'y',
 'y_long',
 'y_mass32',
 'y_obj3',
 'y_obj9']